In [1]:
import glob
import importlib.util
import contextlib
import os
import pandas as pd
import traceback
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import os
import subprocess

def pretty_print(df):
    return display(HTML(df.to_html(escape=False).replace("<td>", "<td style='text-align: left;'>")))

def pie_plot_tests(passed_tests, failed_tests):
    plt.figure(figsize=(8, 8))
    total_tests = passed_tests + failed_tests
    plt.pie([passed_tests, failed_tests], labels=['Passed', 'Failed'], autopct='%1.1f%%', colors=['green', 'red'])
    plt.title('Test Results')
    plt.text(0, 0, f'Total tests: {total_tests}\nPassed: {passed_tests}\nFailed: {failed_tests}', 
             horizontalalignment='center', verticalalignment='center', fontsize=12, bbox=dict(facecolor='white', alpha=0.8))
    plt.show()

def run_tests(test_fn = 'tests'):
    test_files = glob.glob(f'./{test_fn}/*.py')
    failed_tests_df = pd.DataFrame(columns=['File', 'Test', 'Error', 'Traceback'])
    passed_tests = 0
    failed_tests = 0
    html_outputs = []  # list to store HTML outputs
    # For each test file
    for test_file in test_files:
        # Print the file name
        html_outputs.append(HTML(f'<h3>Running tests from {os.path.basename(test_file)}...</h2>'))
        # Define __file__ as the path of the test file
        __file__ = test_file
        # Load the script as a module
        spec = importlib.util.spec_from_file_location('__main__', test_file)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        # Get a list of all test functions (functions whose name starts with 'test_')
        test_functions = [getattr(module, name) for name in dir(module) if name.startswith('test_') and callable(getattr(module, name))]
        # For each test function
        for test_function in test_functions:
            html_outputs.append(HTML(f'<h4><p> &nbsp;&nbsp;&nbsp;&nbsp;Running {test_function.__name__}...</p>'))
            # Try to call the function, suppressing its output
            try:
                test_function()
                passed_tests+=1
                html_outputs.append(HTML(f'<p style="color:green;">&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;Successfully end to ran {test_function.__name__}</p>'))
            except Exception as e:
                failed_tests+=1
                tb = traceback.format_exc()
                html_outputs.append(HTML(f'<p style="color:red;">&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;Error running {test_function.__name__}: {e}</p>'))
                tb_html = tb.replace('\n', '<br>')
                failed_tests_df = pd.concat([failed_tests_df, pd.DataFrame([{'File': test_file, 'Test': test_function.__name__, 'Error': str(e), 'Traceback': tb_html}])], ignore_index=True)
            clear_output(wait=True)  # clear cell output after running each test
    for html_output in html_outputs: display(html_output)
    pie_plot_tests(passed_tests, failed_tests)
    pretty_print(failed_tests_df)
    return failed_tests_df

def process_python_files(directory = 'simpml'):
    """
    Loop through all Python files in the directory tree and run the specified commands on them.

    Parameters:
    - directory (str): The root directory to start the search.
    """
    
    # Create an empty DataFrame to store failures
    failures_df = pd.DataFrame(columns=["File Path", "Command", "Output"])
    
    # Walk through the directory tree
    for root, dirs, files in os.walk(directory):
        # Skip unwanted directories
        dirs[:] = [d for d in dirs if d not in ['.ipynb_checkpoints', '__pycache__']]
        
        # Display the folder name with <h3> tag
        folder_name = os.path.basename(root)
        display(HTML(f"<h3 style='margin-left: {10 * root.count(os.sep)}px'>{folder_name}</h3>"))
        
        for file in files:
            # Check if the file is a Python file
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                
                # Display the file path with <h4> tag
                display(HTML(f"<h4 style='margin-left: {10 * file_path.count(os.sep)}px'>{file}</h4>"))
                
                # Run the commands on the Python file and capture the output
                for command in [["isort", file_path], ["flake8", file_path], ["black", file_path], ["mypy", file_path]]:
                    result = subprocess.run(command, capture_output=True, text=True)
                    
                    # Print the output and success/failure messages
                    if result.returncode == 0:
                        display(HTML(f"<div style='color: green; margin-left: {10 * (file_path.count(os.sep) + 1)}px;'>Success running {command[0]}</div>"))
                    else:
                        display(HTML(f"<div style='color: red; margin-left: {10 * (file_path.count(os.sep) + 1)}px;'>Failure running {command[0]}</div>"))
                        display(HTML(f"<div style='color: red; margin-left: {10 * (file_path.count(os.sep) + 2)}px; white-space: pre-wrap;'>{result.stdout}</div>"))
                        error_data = pd.DataFrame({
                            "File Path": [file_path],
                            "Command": [command[0]],
                            "Output": [result.stdout]
                        })
                        failures_df = pd.concat([failures_df, error_data], ignore_index=True)
    pretty_print(failures_df)
    # Return the failures DataFrame
    return failures_df

In [2]:
failures_df = process_python_files('simpml')

,File Path,Command,Output
0,simpml/tabular/all.py,mypy,
1,simpml/tabular/hyperparameters_optimizer.py,mypy,


In [ ]:
failed_tests_df = run_tests()

,Experiment ID,Model,Description,Data Version,Data Description,Model Params,Metric Params,Accuracy,AUC,Recall,Precision,BalancedAccuracy,F1,Run Time
0,617d0820,Random Baseline Classification,Default settings,57ca5b45,,{'num_epocs': 1},{},0.558187,0.500391,0.336117,0.324597,0.500391,0.330256,0:00:35
1,617d0820,Naive Baseline Classification,Default settings,57ca5b45,,{'num_epocs': 1},{},0.690122,0.555608,0.173278,0.572414,0.555608,0.266026,0:00:39
2,617d0820,Resnet-50,Resnet-50 Pretrained Model ImageNet,57ca5b45,,{'num_epocs': 1},{},0.997294,0.996368,0.993737,0.997904,0.996368,0.995816,0:52:31


epoch,train_loss,valid_loss,time
0,0.140128,0.027588,10:11


epoch,train_loss,valid_loss,time
